In [4]:
import cv2
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time

# Step 1: Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Step 2: Gender Prediction Helper Function
def predict_gender(face_image):
    # Dummy function for gender prediction (replace with actual model for better results)
    return "Male" if np.random.rand() > 0.5 else "Female"

# Step 3: Facial embeddings for identity recognition
# Simple embedding generator (replace with a deep learning model like FaceNet for better accuracy)
def generate_face_embedding(face_image):
    return cv2.resize(face_image, (100, 100)).flatten()

# Step 4: In-Memory Database for Current Session
face_embeddings = []  # Stores face embeddings
face_labels = []      # Stores corresponding labels (e.g., "Person_1")

min_similarity_threshold = 0.8  # Adjust for tighter similarity checks

def recognize_face(embedding):
    for idx, db_embedding in enumerate(face_embeddings):
        similarity = cosine_similarity(embedding.reshape(1, -1), db_embedding.reshape(1, -1))[0][0]
        if similarity > min_similarity_threshold:
            return face_labels[idx]
    return None

# Step 5: Capture Video and Process Faces
cap = cv2.VideoCapture(0)

output_dir = "captured_faces"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Press 'ESC' to quit.")
image_counter = 0
max_images = 5

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    for (x, y, w, h) in faces:
        face_region = gray[y:y+h, x:x+w]

        # Predict gender
        gender = predict_gender(face_region)

        # Generate embedding for the face
        embedding = generate_face_embedding(face_region)

        # Recognize or add new face
        identity = recognize_face(embedding)
        if identity is None:
            # Assign a new identity
            identity = f"Person_{len(face_embeddings) + 1}"
            face_embeddings.append(embedding)
            face_labels.append(identity)
            print(f"New person detected: {identity}")
        else:
            print(f"Recognized: {identity}")

        # Label the face with identity and gender
        label = f"{identity} ({gender})"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # Save captured images if the counter is less than max_images
        if image_counter < max_images:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            image_path = os.path.join(output_dir, f"{label}_{timestamp}_{image_counter}.jpg")
            cv2.imwrite(image_path, frame)
            print(f"Captured image: {image_path}")
            image_counter += 1
            time.sleep(2)
        else:
            print("Maximum of 5 pictures taken. Exiting capture loop.")
            break

    cv2.imshow('Gender and Identity Recognition', frame)

    if cv2.waitKey(1) & 0xFF == 27 or image_counter >= max_images:  # 'ESC' key to exit
        break

cap.release()
cv2.destroyAllWindows()


Press 'ESC' to quit.
New person detected: Person_1
Captured image: captured_faces\Person_1 (Female)_20241205_090909_0.jpg
Recognized: Person_1
Captured image: captured_faces\Person_1 (Male)_20241205_090915_1.jpg
Recognized: Person_1
Captured image: captured_faces\Person_1 (Female)_20241205_090920_2.jpg
Recognized: Person_1
Captured image: captured_faces\Person_1 (Female)_20241205_090926_3.jpg
Recognized: Person_1
Captured image: captured_faces\Person_1 (Male)_20241205_090941_4.jpg
